In [1]:
%matplotlib inline
#%qtconsole

In [2]:
import numpy as np
import scipy.stats
from statsmodels.api import GLM, families
from regularized_glm.core import penalized_IRLS

n_samples = 1000
response = scipy.stats.norm(loc=5).rvs(n_samples)
design_matrix = np.ones_like(response)

statsmodels_fit = GLM(response, design_matrix).fit()

(coefficients, is_converged, coefficient_covariance, aic, deviance,
 effective_degrees_of_freedom, scale) = penalized_IRLS(design_matrix, response)


display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

display(statsmodels_fit.aic)
display(aic)

display(statsmodels_fit.scale)
display(scale)

/Users/edeno/miniconda3/envs/regularized_glm/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


array([5.00448263])

array([5.00448263])

True

array([[0.00094571]])

array([[0.00094571]])

2783.059113039942

2783.059113039942

0.9457117569444125

0.9457117569444125

In [3]:
n_samples = 1000
response = np.concatenate(
    (scipy.stats.norm(loc=5).rvs(n_samples // 2),
     scipy.stats.norm(loc=10).rvs(n_samples // 2)))
design_matrix = np.ones((n_samples, 2))
design_matrix[:n_samples // 2, 1] = 0

statsmodels_fit = GLM(response, design_matrix).fit()

(coefficients, is_converged, coefficient_covariance, aic, deviance,
 effective_degrees_of_freedom, scale) = penalized_IRLS(design_matrix, response)


display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

display(statsmodels_fit.aic)
display(aic)

display(statsmodels_fit.scale)
display(scale)

array([4.90899891, 5.08485492])

array([4.90899891, 5.08485492])

True

array([[ 0.00221295, -0.00221295],
       [-0.00221295,  0.00442589]])

array([[ 0.00221295, -0.00221295],
       [-0.00221295,  0.00442589]])

2941.0526576556263

2941.0526576556263

1.1064731272122632

1.1064731272122632

In [4]:
n_samples = 1000
response = np.concatenate(
    (scipy.stats.norm(loc=5).rvs(n_samples // 2),
     scipy.stats.norm(loc=10).rvs(n_samples // 2)))
design_matrix = np.ones((n_samples, 2))
design_matrix[:n_samples // 2, 1] = 0

statsmodels_fit = GLM(response, design_matrix).fit()

sqrt_penalty_matrix = np.eye(2)
penalty = np.array([0, 1E10])

(coefficients, is_converged, coefficient_covariance, aic, deviance,
 effective_degrees_of_freedom, scale) = penalized_IRLS(
    design_matrix, response, penalty=penalty)

display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

display(statsmodels_fit.aic)
display(aic)

display(statsmodels_fit.scale)
display(scale)

array([4.98118344, 5.00389907])

array([7.48313291e+00, 1.25097474e-07])

True

array([[ 0.00191399, -0.00191399],
       [-0.00191399,  0.00382798]])

array([[7.22205394e-03, 7.22205421e-10],
       [7.22205421e-10, 9.02756788e-17]])

2795.917658547763

4816.015958120528

0.9569947196122957

7.222053942507659

In [5]:
n_samples = 2000
rate = np.ones((n_samples,)) * 12
sampling_frequency = 1000
response = np.random.poisson(rate / sampling_frequency)
design_matrix = np.ones_like(response)

fam = families.Poisson()
statsmodels_fit = GLM(response, design_matrix, family=fam).fit()
(coefficients, is_converged, coefficient_covariance, aic, deviance,
 effective_degrees_of_freedom, scale) = penalized_IRLS(
    design_matrix, response, family=fam)

display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

display(statsmodels_fit.aic)
display(aic)

display(statsmodels_fit.scale)
display(scale)

array([-4.42284863])

array([-4.42284863])

True

array([[0.04166666]])

array([[0.04166667]])

262.2967342013187

262.2967342013185

1.0

1.0

In [6]:
n_samples = 2000
rate = np.ones((n_samples,)) * 12
sampling_frequency = 1000
response = np.random.poisson(rate / sampling_frequency)
design_matrix = np.ones((n_samples, 2))
design_matrix[:n_samples // 2, 1] = 0

fam = families.Poisson()
statsmodels_fit = GLM(response, design_matrix, family=fam).fit()
(coefficients, is_converged, coefficient_covariance, aic, deviance,
 effective_degrees_of_freedom, scale) = penalized_IRLS(
    design_matrix, response, family=fam)

display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

display(statsmodels_fit.aic)
display(aic)

display(statsmodels_fit.scale)
display(scale)

array([-4.34280592,  0.07410797])

array([-4.34280592,  0.07410797])

True

array([[ 0.07692308, -0.07692308],
       [-0.07692308,  0.14835165]])

array([[ 0.07692308, -0.07692308],
       [-0.07692308,  0.14835165]])

290.4364965418082

290.4364965418082

1.0

1.0

In [7]:
n_samples = 2000
rate = np.ones((n_samples,)) * 12
sampling_frequency = 1000
response = np.random.poisson(rate / sampling_frequency)
design_matrix = np.ones((n_samples, 2))
design_matrix[:n_samples // 2, 1] = 0

fam = families.Poisson()
statsmodels_fit = GLM(response, design_matrix, family=fam).fit()

penalty = np.array([0, 1E3])

(coefficients, is_converged, coefficient_covariance, aic, deviance,
 effective_degrees_of_freedom, scale) = penalized_IRLS(
    design_matrix, response, family=fam, penalty=penalty)

display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

display(statsmodels_fit.aic)
display(aic)

display(statsmodels_fit.scale)
display(scale)

array([-4.50986001,  0.37469345])

array([-4.30630798e+00,  2.48323815e-03])

True

array([[ 0.09090909, -0.09090909],
       [-0.09090909,  0.15340909]])

array([[3.70111013e-02, 1.01069887e-03],
       [1.01069887e-03, 3.42646031e-05]])

289.5422499517982

288.47457694939254

1.0

1.0

In [8]:
def simulate_time(n_samples, sampling_frequency):
    return np.arange(n_samples) / sampling_frequency


def simulate_linear_distance(time, track_height, running_speed=10):
    half_height = (track_height / 2)
    return (half_height * np.sin(2 * np.pi * time / running_speed - np.pi / 2)
            + half_height)

def simulate_poisson_spikes(rate, sampling_frequency):
    return 1.0 * (np.random.poisson(rate / sampling_frequency) > 0)


def create_place_field(
    place_field_mean, linear_distance, sampling_frequency, is_condition=None,
        place_field_std_deviation=12.5, max_firing_rate=20,
        baseline_firing_rate=0.5):
    if is_condition is None:
        is_condition = np.ones_like(linear_distance, dtype=bool)
    field_firing_rate = scipy.stats.norm(
        place_field_mean, place_field_std_deviation).pdf(linear_distance)
    field_firing_rate /= field_firing_rate.max()
    field_firing_rate[~is_condition] = 0
    return baseline_firing_rate + max_firing_rate * field_firing_rate

In [9]:
from patsy import build_design_matrices, dmatrices, dmatrix

track_height = 170
n_samples = sampling_frequency * 65

time = simulate_time(n_samples, sampling_frequency)
linear_distance = simulate_linear_distance(
    time, track_height)

place_fields = create_place_field(
    100, linear_distance, sampling_frequency)
spikes = simulate_poisson_spikes(place_fields, sampling_frequency).T

knot_spacing = 15

min_distance, max_distance = (linear_distance.min(),
                              linear_distance.max())
n_steps = (max_distance - min_distance) // knot_spacing
position_knots = min_distance + np.arange(1, n_steps) * knot_spacing
formula = ('is_spike ~ 1 + cr(linear_distance, knots=position_knots,'
           ' constraints="center")')

data = {
    'is_spike': spikes,
    'linear_distance': linear_distance
}
response, design_matrix = dmatrices(formula, data)


In [10]:
%%timeit

penalty = np.ones((design_matrix.shape[1],)) * 1E1
penalty[0] = 0.0

(coefficients, is_converged, coefficient_covariance, aic, deviance,
 effective_degrees_of_freedom, scale) = penalized_IRLS(
    design_matrix, response.squeeze(), family=families.Poisson(), penalty=penalty)

175 ms ± 60.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit

model = GLM(response, design_matrix, family=families.Poisson())

penalty = np.ones((design_matrix.shape[1],)) * 1E1
penalty[0] = 0.0
fit = model.fit_regularized(alpha=penalty, L1_wt=0,
                            maxiter=25)

2.53 s ± 397 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
